<a href="https://colab.research.google.com/github/faizanhemotra/CE888/blob/main/Project/Emotion_XL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data

In [ ]:
import numpy as np
import random
import re
import requests

In [ ]:
def shape_data(feature, label):
    '''Returns number of rows for feature and label'''
    return len(feature.splitlines()), len(label.splitlines())

In [ ]:
emotion_train_url = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_text.txt'
emotion_train = requests.get(emotion_train_url)
emotion_train_labels_url = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_labels.txt'
emotion_train_labels = requests.get(emotion_train_labels_url)
print('Emotion Training, Label')
shape_data(emotion_train.text, emotion_train_labels.text)

Emotion Training, Label


(3257, 3257)

In [ ]:
emotion_val_url = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_text.txt'
emotion_val = requests.get(emotion_val_url)
emotion_val_labels_url = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_labels.txt'
emotion_val_labels = requests.get(emotion_val_labels_url)
print('Emotion Validation, Label')
shape_data(emotion_val.text, emotion_val_labels.text)

Emotion Validation, Label


(374, 374)

In [ ]:
emotion_test_url = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt'
emotion_test = requests.get(emotion_test_url)
emotion_test_labels_url = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt'
emotion_test_labels = requests.get(emotion_test_labels_url)
print('Emotion Test, Label')
shape_data(emotion_test.text, emotion_test_labels.text)

Emotion Test, Label


(1421, 1421)

##Labels

In [ ]:
emotion_train_distr = list()
for label in emotion_train_labels.text.splitlines():
    emotion_train_distr.append(int(label))

In [ ]:
emotion_val_distr = list()
for label in emotion_val_labels.text.splitlines():
    emotion_val_distr.append(int(label))
    
emotion_test_distr = list()
for label in emotion_test_labels.text.splitlines():
    emotion_test_distr.append(int(label))

In [ ]:
labels = list(set(emotion_train_distr))
print(labels)

[0, 1, 2, 3]


In [ ]:
def linkremover(text):
    '''Removes t.co and instagram.com links from the data'''
    tco = r'https?://t.co/[\w]{10}'
    insta = r'https?://(www.)?instagram.com/p/[-\w]{11}/?'
    if (re.findall(tco + '|' + insta, text) != []):
        final = re.sub(tco + '|' + insta, '', text)
        return final
    else:
        print('No links')
    return text

In [ ]:
emotion_tr = linkremover(emotion_train.text)
emotion_v = linkremover(emotion_val.text)
emotion_te = linkremover(emotion_test.text)

No links
No links
No links


In [ ]:
import os
import torch
hftransformers = os.path.exists('/content/transformers')
if hftransformers:
  print('transformers already downloaded ')
else:
  !git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 70267, done.
remote: Counting objects: 100% (358/358), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 70267 (delta 193), reused 219 (delta 117), pack-reused 69909
Receiving objects: 100% (70267/70267), 53.57 MiB | 29.19 MiB/s, done.
Resolving deltas: 100% (49761/49761), done.


In [ ]:
cd transformers

/content/transformers


In [ ]:
if hftransformers:
  print('transformers already downloaded ')
else:
  !pip3 install --upgrade .

Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 18.4MB/s 
     |████████████████████████████████| 901kB 49.6MB/s 
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2112655 sha256=c14dc61a34a4c45290791c486f8230270a569f5043135f9c30eab3299f559c2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-x5ogzzn7/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers


In [ ]:
import importlib
emoji = importlib.util.find_spec("emoji")
found = emoji is not None
if found:
  print('emoji package already exists')
else:
  !pip3 install emoji

     |████████████████████████████████| 133kB 18.0MB/s 


## Tokenizing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlnet-base-cased', use_fast=False)

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
train_encodings = tokenizer(emotion_tr.splitlines(), truncation=True, padding=True)
valid_encodings = tokenizer(emotion_v.splitlines(), truncation=True, padding=True)
test_encodings = tokenizer(emotion_te.splitlines(), truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
class Emotion(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = Emotion(train_encodings, emotion_train_distr)
valid_dataset = Emotion(valid_encodings, emotion_val_distr)
test_dataset = Emotion(test_encodings, emotion_test_distr)

##Training

In [ ]:
from sklearn.metrics import f1_score
from transformers import XLNetForSequenceClassification, Trainer, TrainingArguments

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='macro')
    return {
        'f1': f1
    }

model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', cache_dir='cache/', num_labels=len(labels))
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total # of training epochs
    per_device_train_batch_size=12,  # batch size per device during training
    per_device_eval_batch_size=12,   # batch size for evaluation
    evaluation_strategy='epoch',
    disable_tqdm=False,
    load_best_model_at_end=True,
    save_strategy='epoch',
    save_total_limit=7,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=1e-5,
    logging_steps=8,
    logging_dir='./logs',            # directory for storing logs
    dataloader_num_workers=2,
    run_name='xlnet-classification'
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
if device == 'cuda':
  !nvidia-smi

Tue Apr 20 10:59:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    29W /  70W |   1548MiB / 15109MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
trainer.train()

TrainOutput(global_step=1360, training_loss=0.7181325974709848, metrics={'train_runtime': 463.5531, 'train_samples_per_second': 2.934, 'total_flos': 825304025580480.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 2333724672, 'init_mem_gpu_alloc_delta': 469510656, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 626688, 'train_mem_gpu_alloc_delta': 1446875648, 'train_mem_cpu_peaked_delta': 20914176, 'train_mem_gpu_peaked_delta': 883569664})

In [ ]:
torch.save(model.state_dict(), 'model_xlnet_emotion.pth')

## Evaluations

In [ ]:
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
def plot_confusion_matrix(conf_mat, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    conf_mat: confusion matrix
    classes: names of classes to be annotated
    normalize: [True/False] whether to normalise by column
    """
    if normalize:
        conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.imshow(conf_mat, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    for i, j in itertools.product(range(conf_mat.shape[0]), range(conf_mat.shape[1])):
        t = "(%.2f)" % (conf_mat[i, j])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### Validation Dataset

In [ ]:
val_pred = trainer.predict(valid_dataset)
val_pred.metrics

{'test_f1': 0.7145704832292634,
 'test_loss': 0.6773463487625122,
 'test_mem_cpu_alloc_delta': 0,
 'test_mem_cpu_peaked_delta': 0,
 'test_mem_gpu_alloc_delta': 0,
 'test_mem_gpu_peaked_delta': 72060416,
 'test_runtime': 2.4307,
 'test_samples_per_second': 153.863}

In [ ]:
validation_pred = []
for i in range(len(val_pred.predictions)):
  validation_pred.append(np.argmax(val_pred.predictions[i]))

In [ ]:
cnf_matrix = confusion_matrix(emotion_val_distr, validation_pred)
plot_confusion_matrix(cnf_matrix, classes=['anger', 'joy', 'optimism', 'sadness'], normalize=True,
                      title='Validation Dataset Confusion matrix')

Normalized confusion matrix


In [ ]:
print(classification_report(emotion_val_distr, validation_pred, digits=5))

              precision    recall  f1-score   support

           0    0.81034   0.88125   0.84431       160
           1    0.74194   0.71134   0.72632        97
           2    0.68421   0.46429   0.55319        28
           3    0.73864   0.73034   0.73446        89

    accuracy                        0.77005       374
   macro avg    0.74378   0.69680   0.71457       374
weighted avg    0.76609   0.77005   0.76577       374



### Test Dataset

In [ ]:
te_pred = trainer.predict(test_dataset)
te_pred.metrics

{'test_f1': 0.7697686820410097,
 'test_loss': 0.6031834483146667,
 'test_mem_cpu_alloc_delta': 98304,
 'test_mem_cpu_peaked_delta': 0,
 'test_mem_gpu_alloc_delta': 0,
 'test_mem_gpu_peaked_delta': 108081664,
 'test_runtime': 9.9987,
 'test_samples_per_second': 142.118}

In [ ]:
test_pred = []
for i in range(len(te_pred.predictions)):
  test_pred.append(np.argmax(te_pred.predictions[i]))

In [ ]:
cnf_matrix = confusion_matrix(emotion_test_distr, test_pred)
plot_confusion_matrix(cnf_matrix, classes=['anger', 'joy', 'optimism', 'sadness'], normalize=True,
                      title='Test Dataset Confusion matrix')

Normalized confusion matrix


In [ ]:
print(classification_report(emotion_test_distr, test_pred, digits=5))

              precision    recall  f1-score   support

           0    0.81895   0.86738   0.84247       558
           1    0.80563   0.79888   0.80224       358
           2    0.85714   0.53659   0.66000       123
           3    0.75879   0.79058   0.77436       382

    accuracy                        0.80084      1421
   macro avg    0.81013   0.74836   0.76977      1421
weighted avg    0.80273   0.80084   0.79823      1421



## Saving to Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !cp -r ./model_emotion.pth /content/drive/MyDrive